In [ ]:
# Import libraries
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

from tqdm.auto import trange

import numpy as np

from scipy.linalg import eigvals

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import pandas as pd

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

## Generate quantum states
Generate *N_states* separable and entangled states and eventually save them

Define the functions to generate the states and check that they are entangled

In [ ]:
def generate_hermitian_product_states(size, n_matrices):
    """Generates a list of random Hermitian product states
    of the given dimension
    Product states are hermitian matrices with trace 1.
    Input:
        size: size of the matrices
        n_matrices: number of matrices to generate
    Output:
        product_states: 3D numpy array of product states
    """

    product_states = []
    for _ in range(n_matrices):
        real_part = np.random.rand(size, size)
        imag_part = np.random.rand(size, size)
        product_state = real_part + 1j * imag_part
        product_state = np.matmul(product_state, product_state.conj().T)
        product_state /= np.trace(product_state)
        product_states.append(product_state)

    return np.array(product_states)


def generate_coefficients(n):
    """
    Generates a list of n random coefficients that sum to 1
    Input:
        n: number of coefficients to generate
    Output:
        coefficients: numpy array of n coefficients
    """
    rand_numbers = np.random.rand(n)
    rand_numbers /= sum(rand_numbers)

    return rand_numbers


def generate_separable_states(n_matrices, n_states):
    """Generates a list of random separable states of the given dimension
    Input:
        dimensions: size of the matrices
        n_matrices: number of matrices used to generate the states
        n_states: number of separable states to generate
    Output:
        separable_states: 3D numpy array of separable states,
        of size n_states x dimensions^n_matrices
    """

    states = []

    for _ in range(n_states):
        rhoA = generate_hermitian_product_states(2, n_matrices)
        rhoB = generate_hermitian_product_states(2, n_matrices)
        coeffs = generate_coefficients(n_matrices)

        sep_state = 0

        for i in range(n_matrices):
            sep_state += coeffs[i] * np.kron(rhoA[i], rhoB[i])

        states.append(sep_state)

    return th.tensor(states, dtype=th.complex64)


# Entanglement check with Peres-Horodecki criterion
def is_entangled(rho):
    # Check if the density matrix is 4x4
    if rho.shape != (4, 4):
        raise ValueError("The input matrix should be a 4x4 density matrix.")

    # Calculate the partial transpose of the density matrix
    pauli_mat_B = np.array([[1, 0], [0, -1]])
    identity_mat = np.eye(2)
    
    transpose_op = np.kron(identity_mat, pauli_mat_B)
    rho_T_B = transpose_op @ np.transpose(rho) @ transpose_op
    eigenvalues = eigvals(rho_T_B)

    return any(eig < 0 for eig in eigenvalues)


def generate_entangled_states(n_states):

    states = []
    i = 0
    while i < n_states:
        rand_state = np.random.rand(4, 4)

        if is_entangled(rand_state):
            states.append(rand_state)
            i += 1
        else:
            continue
    return th.tensor(states, dtype=th.complex64)


In [ ]:
N_states_train = 10000
sep_data = generate_separable_states(10, N_states_train)
ent_data = generate_entangled_states(N_states_train)

Check if the elemnts of the datasets are truly all separable and entangled

In [ ]:
print("Any entangled data in the separable dataset?: ", any(is_entangled(sep_data[i].numpy()) for i in range(N_states_train)))
print("Are all the data in the entangled dataset entangled?: ", all(is_entangled(ent_data[i].numpy()) for i in range(N_states_train)))

Reshape the dataset into a *N_states* $\times\, (4 * 4 * 2)$ matrix and create a data loader

In [ ]:
def data_reshape(dataset):
    """Reshapes the data to be used in the neural network
    Input:
        dataset: dataset of states to reshape
    Output:
        reshaped_dataset: dataset of states reshaped to be used in the neural network
    """

    dataset_reshaped = th.empty(dataset.shape[0], 2 * dataset.shape[1] * dataset.shape[2])
    
    for i in range(dataset.shape[0]):
        dataset_reshaped[i] = th.cat((dataset[i].real.flatten(), dataset[i].imag.flatten()), dim = 0)
    
    return dataset_reshaped

In [ ]:
full_data_sep = data_reshape(sep_data)
full_data_ent = data_reshape(ent_data)

BATCH_SIZE = 64
sep_train_loader = DataLoader(full_data_sep, batch_size=BATCH_SIZE, shuffle=True)
ent_train_loader = DataLoader(full_data_ent, batch_size=BATCH_SIZE, shuffle=True)

Create a test set

In [ ]:
N_states_test = 3000

sep_data = generate_separable_states(10, N_states_test)
ent_data = generate_entangled_states(N_states_test)

print("Any entangled data in the separable dataset?: ", any(is_entangled(sep_data[i].numpy()) for i in range(N_states_test)))
print("Are all the data in the entangled dataset entangled?: ", all(is_entangled(ent_data[i].numpy()) for i in range(N_states_test)))

In [ ]:
full_data_sep = data_reshape(sep_data)
full_data_ent = data_reshape(ent_data)

sep_test_loader = DataLoader(full_data_sep, batch_size=BATCH_SIZE, shuffle=True)
ent_test_loader = DataLoader(full_data_ent, batch_size=BATCH_SIZE, shuffle=True)

## PCA
Perform PCA to analyze the dataset, with the following steps:
1. Merge the two datasets into a single one of shape $(2 *$*N_states*$) \times 32$
2. Assign to each row a label: 0 if it represents a separable state, 1 if it represents and entangled one
3. Perform PCA and plot the first two principal components and the explained variance ratio

In [ ]:
# Merge the two datasets
full_dataset = np.concatenate((full_data_sep, full_data_ent), axis=0)

# Create the labels
label_sep = th.zeros(full_data_sep.shape[0])
label_ent = th.ones(full_data_sep.shape[0])
labels = th.cat([label_sep, label_ent], dim=0).detach().numpy()

In [ ]:
# Perform PCA

pca_model = PCA()

pca = pca_model.fit(full_dataset)

In [ ]:
# Calculate the principal components
pca_components = pca.transform(full_dataset)

# Plot the first two principal components
scatter = plt.scatter(pca_components[:, 0], pca_components[:, 1], c=labels)
unique_labels = np.unique(labels)
unique_colors = [scatter.cmap(scatter.norm(label)) for label in unique_labels]
label_names = ['Separable', 'Entangled']
label_name_dict = dict(zip(unique_labels, label_names))
patches = [mpatches.Patch(color=unique_colors[i], label=label_name_dict[label]) for i, label in enumerate(unique_labels)]
plt.legend(handles=patches)

In [ ]:
# Calculate the explained variance ratio
exp_variance = pca.explained_variance_ratio_

# Plot the explained variance ratio
fig = plt.plot(exp_variance, linestyle = '-', marker = '.')
plt.xticks(np.arange(1, len(exp_variance) + 1, 2))
plt.xlabel("Principal components")
plt.ylabel("Explained variance ratio")

Check how many principal components have an explained variance ratio greater than a specific threshold. This will be the dimension of the latent space of the VAE.

In [ ]:
threshold = 0.01

latent_space_dim = 0
for var in exp_variance:
    if var > threshold:
        latent_space_dim += 1
        
latent_space_dim

## Define the Variational Autoencoder

In [ ]:
class VAE_fc(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(VAE_fc, self).__init__()
        self.input_size = input_size
        
        # Define the encoder layers
        self.enc1 = nn.Linear(input_size, hidden_size[0])
        self.enc2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.enc_mu = nn.Linear(hidden_size[1], hidden_size[2])
        self.enc_logvar = nn.Linear(hidden_size[1], hidden_size[2])
        
        # Define the decoder layers
        self.dec1 = nn.Linear(hidden_size[2], hidden_size[1])
        self.dec2 = nn.Linear(hidden_size[1], hidden_size[0])
        self.dec3 = nn.Linear(hidden_size[0], input_size)
        
        # Define the activation function
        self.relu = nn.ReLU()
    
    # Define the encoder
    def encoder(self, x):
        x = self.relu(self.enc1(x))
        x = self.relu(self.enc2(x))
        mu = self.enc_mu(x)
        logvar = self.enc_logvar(x)
        return mu, logvar
    
    # Define the normal distribution
    def reparametrize(self, mu, logvar):
        std = th.exp(0.5*logvar)
        eps = th.randn_like(std)
        return mu + eps*std
    
    # Define the decoder
    def decoder(self, z):
        z = self.relu(self.dec1(z))
        z = self.relu(self.dec2(z))
        z = self.relu(self.dec3(z))
        return z
    
    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparametrize(mu, logvar)
        decoded = self.decoder(z)
        # Return the output of the decoder, the latent space vector, and the mean and logvar of the normal distribution
        return decoded, z, mu, logvar

In [ ]:
# Define the loss function
def loss_function(output, x, mu, logvar):
    recon_loss = F.mse_loss(output, x, reduction='sum')
    kl_loss = -0.5 * th.sum(1 + logvar - mu * mu - logvar.exp())
    return recon_loss + 0.5 * kl_loss

In [ ]:
# Define a function to train the model
def train_model(model, train_loader, optimizer, criterion, epochs):
    
    train_loss = []
    for epoch in trange(epochs):
        epoch_loss = 0.0
        
        for _, data in enumerate(train_loader):
            optimizer.zero_grad()
            output, _, mu, logvar = model(data)
            loss = criterion(output, data, mu, logvar)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            
        epoch_loss /= len(train_loader.dataset)
        train_loss.append(epoch_loss)
        print(f"Epoch: {epoch}, Loss: {epoch_loss:.4f}")

    return model, train_loss

In [ ]:
# Define constant parameters
HIDDEN_SIZE = [50, 25, latent_space_dim]
EPOCHS = 15
LR = 0.001
INPUT_SIZE = n_columns

In [ ]:
# Train VAE on the separable dataset
model = VAE_fc(input_size = INPUT_SIZE, hidden_size = HIDDEN_SIZE)
optimizer = optim.Adam(model.parameters(), lr = LR)

sep_trained_model, sep_train_loss = train_model(model, sep_train_loader, optimizer, loss_function, EPOCHS)

In [ ]:
# Train entangled VAE
model = VAE_fc(input_size = INPUT_SIZE, hidden_size = HIDDEN_SIZE)
optimizer = optim.Adam(model.parameters(), lr = LR)

ent_trained_model, ent_train_loss = train_model(model, ent_train_loader, optimizer, loss_function, EPOCHS)

Plot the losses for each epoch and their difference

In [ ]:
plt.plot(sep_train_loss, label='Separable')
plt.plot(ent_train_loss, label='Entangled')
plt.plot(np.abs(np.array(sep_train_loss) - np.array(ent_train_loss)), label='Difference')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')

Calculate the reconstruction loss on the test sets

In [ ]:
test_loss = np.ndarray((2, 2))


for i, data in enumerate([ent_test_loader.dataset, sep_test_loader.dataset]):
    for j, model in enumerate([ent_trained_model, sep_trained_model]):
        test_loss[i, j] = F.mse_loss(model(data)[0], data, reduction='sum').detach().numpy() / len(ent_test_loader.dataset)

test_loss

In [ ]:
test_losses = pd.DataFrame(test_loss, columns=['ent_model', 'sep_model'], index=['ent_set', 'sep_set'])

test_losses

### Check if the two VAE generate the correct kinds of quantum states

Define a function to bring the output in the form of a $4 \times 4$ complex matrix

In [ ]:
def reconstruct_decoded_data(decoded_data):
    reconstructed_data = np.ndarray((len(decoded_data), 4, 4), dtype=complex)

    for i in range(len(decoded_data)):
        decoded = decoded_data[i, 0:16] + 1j * decoded_data[i, 16:]
        reconstructed_data[i] = decoded.reshape((4, 4))
        
    return reconstructed_data

Calculate the outputs of the two autoencoders for both the test sets and reshape them

In [ ]:
decoded_ent_model_ent_data = ent_trained_model(ent_test_loader.dataset)[0].detach().numpy()
decoded_ent_model_sep_data = ent_trained_model(sep_test_loader.dataset)[0].detach().numpy()

decoded_sep_model_sep_data = sep_trained_model(sep_test_loader.dataset)[0].detach().numpy()
decoded_sep_model_ent_data = sep_trained_model(ent_test_loader.dataset)[0].detach().numpy()


reconstructed_ent_model_ent_data = reconstruct_decoded_data(decoded_ent_model_ent_data)
reconstructed_ent_model_sep_data = reconstruct_decoded_data(decoded_ent_model_sep_data)

reconstructed_sep_model_sep_data = reconstruct_decoded_data(decoded_sep_model_sep_data)
reconstructed_sep_model_ent_data = reconstruct_decoded_data(decoded_sep_model_ent_data)

Calculate the \% of entangled and separable states in each output

In [ ]:
def check_data(decoded):
    decoded_sep = 0
    decoded_ent = 0
    for i in range(len(decoded)):
        entangled = is_entangled(decoded[i])
        
        if entangled:
            decoded_ent += 1
        else:
            decoded_sep += 1
    const = 100 / (decoded_sep + decoded_ent)
    print("Entangled data:", decoded_ent * const, "%")
    print("Separable data:", decoded_sep * const, "%")

In [ ]:
print("Entangled model, entangled data")
check_data(reconstructed_ent_model_ent_data)
print()

print("Entangled model, separable data")
check_data(reconstructed_ent_model_sep_data)
print()

print("Separable model, separable data")
check_data(reconstructed_sep_model_sep_data)
print()

print("Separable model, entangled data")
check_data(reconstructed_sep_model_ent_data)
print()

## Distinguish between separable and entangled states using the VAE
1. Generate some labeled states that are entangled or separable at random

In [ ]:
N_states_test = 100

true_labels = np.ndarray((N_states_test, 1))
states = th.empty((N_states_test, 32), dtype=th.float32)

for i in range(N_states_test):
    # Choose randomly beween 0 and 1
    true_labels[i] = np.random.randint(0, 2)
    
    if true_labels[i] == 0:
        # Generate separable state
        state = generate_separable_states(10, 1)
    else:
        # Generate entangled state
        state = generate_entangled_states(1)
    
    # Flatten the state and concatenate the real and imaginary parts
    states[i] = th.cat((state.flatten(start_dim=0).real, state.flatten(start_dim=0).imag), dim=0)

Assing the labelt to the two states based on their loss

In [ ]:
predicted_labels = np.ndarray((N_states_test, 1))

for i in range(N_states_test):
    # Calculate the output of the models
    output_sep = sep_trained_model(states[i])[0]
    output_ent = ent_trained_model(states[i])[0]
    
    # Calculate the loss
    loss_sep = F.mse_loss(output_sep, states[i], reduction='sum').detach().numpy()
    loss_ent = F.mse_loss(output_ent, states[i], reduction='sum').detach().numpy()
    
    # If the loss obtained with the separable model is smaller, then the state is separable
    if loss_sep < loss_ent:
        predicted_labels[i] = 0 
    # Otherwise, the state is entangled
    else:
        predicted_labels[i] = 1 
    

In [ ]:
acc = accuracy_score(true_labels, predicted_labels)

print(f"Accuracy of the predicted labels: {acc*100:.1f}%")